# Simple MNIST convnet

**Author:** [fchollet](https://twitter.com/fchollet)<br>
**Date created:** 2015/06/19<br>
**Last modified:** 2020/04/21<br>
**Description:** A simple convnet that achieves ~99% test accuracy on MNIST.

## Setup

In [61]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os
from google.colab import drive

drive.mount('/Pests')

Drive already mounted at /Pests; to attempt to forcibly remount, call drive.mount("/Pests", force_remount=True).


## Prepare the data

In [62]:
pathtrain = '/Pests/MyDrive/Pests/train'
pathtest ='/Pests/MyDrive/Pests/test'


# Model / data parameters
num_classes = 12
input_shape = (32, 32, 1)



# Load the data and split it between train and test sets
data_train = keras.utils.image_dataset_from_directory(
        pathtrain,
        color_mode="grayscale",
        validation_split=0.2,
        subset="training",
        seed=69,
        image_size=(32, 32),
        batch_size=128)

data_val = keras.utils.image_dataset_from_directory(
        pathtrain,
        color_mode="grayscale",
        validation_split=0.2,
        subset="validation",
        seed=69,
        image_size=(32, 32),
        batch_size=128)


data_test = keras.utils.image_dataset_from_directory(
        pathtest,
        color_mode="grayscale",
        image_size=(32, 32))

def process(image,label):
    image = tf.cast(image/255. ,tf.float32)
    return image,label

data_train = data_train.map(process)
data_val = data_val.map(process)
data_test = data_test.map(process)



Found 4912 files belonging to 12 classes.
Using 3930 files for training.
Found 4912 files belonging to 12 classes.
Using 982 files for validation.
Found 600 files belonging to 12 classes.


## Build the model

In [113]:
model = keras.Sequential(
    [
        keras.Input(shape=input_shape),
 #       layers.Rescaling(1./255),
        layers.Conv2D(64, kernel_size=(4, 4), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(102, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Flatten(),
        layers.Dropout(0.3),
        layers.Dense(num_classes, activation="softmax"),
    ]
)

model.summary()

Model: "sequential_31"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_62 (Conv2D)          (None, 29, 29, 64)        1088      
                                                                 
 max_pooling2d_62 (MaxPooli  (None, 14, 14, 64)        0         
 ng2D)                                                           
                                                                 
 conv2d_63 (Conv2D)          (None, 12, 12, 102)       58854     
                                                                 
 max_pooling2d_63 (MaxPooli  (None, 6, 6, 102)         0         
 ng2D)                                                           
                                                                 
 flatten_31 (Flatten)        (None, 3672)              0         
                                                                 
 dropout_31 (Dropout)        (None, 3672)            

## Train the model

In [114]:
epochs = 20

model.compile(loss=keras.losses.SparseCategoricalCrossentropy(), optimizer="adam", metrics=["accuracy"])
model.fit(
    data_train,
    validation_data=data_val,
    epochs=epochs)


Epoch 1/20
31/31 [==============================] - 15s 403ms/step - loss: 2.4708 - accuracy: 0.1247 - val_loss: 2.4466 - val_accuracy: 0.1497
Epoch 2/20
31/31 [==============================] - 16s 477ms/step - loss: 2.3779 - accuracy: 0.1891 - val_loss: 2.3369 - val_accuracy: 0.1874
Epoch 3/20
31/31 [==============================] - 14s 380ms/step - loss: 2.2668 - accuracy: 0.2257 - val_loss: 2.2607 - val_accuracy: 0.2210
Epoch 4/20
31/31 [==============================] - 16s 473ms/step - loss: 2.1863 - accuracy: 0.2616 - val_loss: 2.2263 - val_accuracy: 0.2393
Epoch 5/20
31/31 [==============================] - 16s 447ms/step - loss: 2.1393 - accuracy: 0.2822 - val_loss: 2.1836 - val_accuracy: 0.2546
Epoch 6/20
31/31 [==============================] - 16s 429ms/step - loss: 2.1029 - accuracy: 0.2941 - val_loss: 2.1521 - val_accuracy: 0.2872
Epoch 7/20
31/31 [==============================] - 14s 394ms/step - loss: 2.0331 - accuracy: 0.3193 - val_loss: 2.1251 - val_accuracy: 0.2933

## Evaluate the trained model

In [115]:
score = model.evaluate(data_test, verbose=0)
print("Test loss:", score[0])
print("Test accuracy:", score[1])

Test loss: 2.1669962406158447
Test accuracy: 0.30666667222976685
